In [9]:
#import the MeTA python bindings
import metapy
#If you'd like, you can tell MeTA to log to stderr so you can get progress output 
#when running long-running function calls.
metapy.log_to_stderr()


doc = metapy.index.Document()
# doc.content("I said that I can't believe that it only costs $19.95!")
doc.content("My Father is Joyful ; A Variant of Abby")

tok = metapy.analyzers.ICUTokenizer(suppress_tags=True)
tok.set_content(doc.content()) # this could be any string
tokens = [token for token in tok]
print("\n\nversion0:",tokens)


tok = metapy.analyzers.LengthFilter(tok, min=2, max=30)
tok.set_content(doc.content()) # this could be any string
tokens = [token for token in tok]
print("\n\nversion1:",tokens)


tok = metapy.analyzers.ListFilter(tok, "lemur-stopwords.txt", metapy.analyzers.ListFilter.Type.Reject)
tok.set_content(doc.content())
tokens = [token for token in tok]
print("\n\nversion2:",tokens)




tok = metapy.analyzers.Porter2Filter(tok)
tok.set_content(doc.content())
tokens = [token for token in tok]
print("\n\nversion3:",tokens)







version0: ['My', 'Father', 'is', 'Joyful', ';', 'A', 'Variant', 'of', 'Abby']


version1: ['My', 'Father', 'is', 'Joyful', 'Variant', 'of', 'Abby']


version2: ['My', 'Father', 'Joyful', 'Variant', 'Abby']


version3: ['My', 'Father', 'Joy', 'Variant', 'Abbi']


In [12]:
s = ""
for t in tokens:
    s += (t+" ")
print(s)

My Father Joy Variant Abbi 


In [14]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

# nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]



0.9999999999999998
0.7092972666062738
0.0


In [43]:
enMeaning = "The joy or happiness of Yahweh"
enMeaning2="My Father Joy Variant Abbi"
enMeaning3="My brother is sad and unhappy"
enMeaning4="My brother feels delighted"


chMeaning = "Happy"
print(cosine_sim(chMeaning,enMeaning))
print(cosine_sim(chMeaning,enMeaning2))
print(cosine_sim(chMeaning,enMeaning3))
print(cosine_sim(chMeaning,enMeaning4))

0.4494364165239821
0.0
0.0
0.0


In [25]:
finalWords = []
for t in tokens:
    finalWords.append(t)
print(type(finalWords))
print(type(finalWords[1]))

for word in finalWords:
    print(word, cosine_sim("Happy",word))





<class 'list'>
<class 'str'>
My 0.0
Father 0.0
Joy 0.0
Variant 0.0
Abbi 0.0


In [32]:
import spacy

nlp = spacy.load('en_core_web_lg')  # make sure to use larger model!
tokens = nlp(u'My Father Joy Variant Abbi')


for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))
        
        

dog dog 1.0
dog cat 0.80168545
dog banana 0.24327646
cat dog 0.80168545
cat cat 1.0
cat banana 0.2815437
banana dog 0.24327646
banana cat 0.2815437
banana banana 1.0


In [41]:
enMeaning = nlp(u"The joy or happiness of Yahweh")
enMeaning2=nlp(u"My Father Joy Variant Abbi")

enMeaning3=nlp(u"My brother is sad and unhappy")
chMeaning = nlp(u"Happy")

print(chMeaning.similarity(enMeaning))
print(chMeaning.similarity(enMeaning2))
print(chMeaning.similarity(enMeaning3))

0.5402151164088576
0.5493037434707008
0.7277400476317061
